In [39]:

from mesa import Agent, Model
from mesa.time import SimultaneousActivation
from mesa.space import NetworkGrid
from mesa.datacollection import DataCollector
import networkx as nx
import numpy as np
import random
from typing import Iterable, List, Dict
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor, as_completed
import os


In [41]:
####################################
# Strategy selection helpers
#
# We provide two different ways agents can choose their strategy:
# - `choose_strategy_imitate`: choose strategy of the highest-payoff neighbour (including self).
# - `choose_strategy_logit`: choose strategy using logit / softmax choice.
#
####################################
def choose_strategy_imitate(agent, neighbors):
    """Choose strategy of the highest-payoff neighbour (including self)."""
    candidates = neighbors + [agent]
    best = max(candidates, key=lambda a: a.payoff)
    return best.strategy

def choose_strategy_logit(agent, neighbors, a_I, b, tau):
    """Choose strategy using logit / softmax choice.

    Parameters
    - agent: the agent choosing a strategy
    - neighbors: list of neighbour agents
    - a_I: effective coordination payoff given current infrastructure
    - b: defection payoff
    - tau: temperature parameter for softmax
    """
    # compute expected payoffs for C and D
    pi_C = 0.0
    pi_D = 0.0
    for other in neighbors:
        s_j = other.strategy
        if s_j == "C":
            pi_C += a_I
            pi_D += b
        else:
            pi_C += 0.0
            pi_D += b

    # softmax choice
    denom = np.exp(pi_C / tau) + np.exp(pi_D / tau)
    P_C = np.exp(pi_C / tau) / denom if denom > 0 else 0.5
    return "C" if random.random() < P_C else "D"



In [42]:
class EVAgent(Agent):
    """Agent for EV Stag Hunt model (SimultaneousActivation-friendly).

    - step(): compute payoff AND decide next_strategy (do NOT commit)
    - advance(): commit next_strategy -> strategy
    """

    def __init__(self, unique_id, model, init_strategy="D"):
        super().__init__(unique_id, model)
        self.strategy = init_strategy
        self.payoff = 0.0
        self.next_strategy = init_strategy

    def _get_neighbor_agents(self):
        """Return list of agent objects that are neighbors on the network grid."""
        neighbors = []
        # self.pos is the node id in NetworkGrid
        for nbr in self.model.G.neighbors(self.pos):
            neighbors.extend(self.model.grid.get_cell_list_contents([nbr]))
        return neighbors

    def step(self):
        """Compute payoff from neighbors and choose next_strategy (but do NOT commit)."""
        I = self.model.infrastructure
        a0 = self.model.a0
        beta_I = self.model.beta_I
        b = self.model.b
        a_I = a0 + beta_I * I

        neighbor_agents = self._get_neighbor_agents()

        # If no neighbors: no interactions, keep payoff 0 and keep current strategy
        if not neighbor_agents:
            self.payoff = 0.0
            self.next_strategy = self.strategy
            return

        # Compute payoff (sum over pairwise interactions)
        payoff = 0.0
        for other in neighbor_agents:
            s_i = self.strategy
            s_j = other.strategy
            if s_i == "C" and s_j == "C":
                payoff += a_I
            elif s_i == "C" and s_j == "D":
                payoff += 0.0
            elif s_i == "D" and s_j == "C":
                payoff += b
            else:  # D vs D
                payoff += b
        self.payoff = payoff

        # Decide next strategy (do NOT assign to self.strategy here)
        func = getattr(self.model, "strategy_choice_func", "imitate")
        if func == "imitate":
            # expected signature: choose_strategy_imitate(agent, neighbor_agents) -> "C" or "D"
            self.next_strategy = choose_strategy_imitate(self, neighbor_agents)
        elif func == "logit":
            tau = getattr(self.model, "tau", 1.0)
            # expected signature: choose_strategy_logit(agent, neighbor_agents, a_I, b, tau)
            self.next_strategy = choose_strategy_logit(self, neighbor_agents, a_I, b, tau)
        else:
            raise ValueError(f"Unknown strategy choice function: {func}")

    def advance(self):
        """Commit the previously chosen next_strategy (synchronous update)."""
        # simply commit choice (no recomputation)
        self.strategy = self.next_strategy


In [46]:

    ####################################
    # Model class
    #
    # The EVStagHuntModel class implements the Mesa model for EV Stag Hunt on a network.
    #
    # Parameters
    # - initial_ev: number of initial EV nodes
    # - a0: base payoff for EV adoption
    # - beta_I: payoff enhancement factor for EV adoption
    # - b: payoff for ICE defection
    # - g_I: infrastructure growth rate
    # - I0: initial infrastructure level
    # - seed: random seed for reproducibility
    # - network_type: type of network to generate ("random" or "BA")
    # - n_nodes: number of nodes in the network
    # - p: probability of edge creation in random network
    # - m: number of edges to attach to new node in BA network
    # - collect: whether to collect agent and model-level data
    # - strategy_choice_func: strategy selection function ("imitate" or "logit")
    # - tau: temperature parameter for softmax choice (only used with "logit")
    ####################################
#
class EVStagHuntModel(Model):
    """Mesa model for EV Stag Hunt on a network."""

    def __init__(
        self,
        initial_ev=10,
        a0=2.0,
        beta_I=3.0,
        b=1.0,
        g_I=0.1,
        I0=0.05,
        seed=None,
        network_type="random",
        n_nodes=100,
        p=0.05,
        m=2,
        collect=True,
        strategy_choice_func: str = "imitate",
        tau: float = 1.0,
    ):
        super().__init__(seed=seed)

        # Build graph
        if network_type == "BA":
            G = nx.barabasi_albert_graph(n_nodes, m, seed=seed)
        else:
            G = nx.erdos_renyi_graph(n_nodes, p, seed=seed)
        self.G = G
        self.grid = NetworkGrid(G)
        self.schedule = SimultaneousActivation(self)

        # parameters
        self.a0 = a0
        self.beta_I = beta_I
        self.b = b
        self.g_I = g_I
        self.infrastructure = I0
        self.step_count = 0
        self.strategy_choice_func = strategy_choice_func
        self.tau = tau

        # initialize node attribute for agent reference
        for n in self.G.nodes:
            self.G.nodes[n]["agent"] = []

        # choose initial EV nodes
        total_nodes = self.G.number_of_nodes()
        k_ev = max(0, min(initial_ev, total_nodes))
        ev_nodes = set(self.random.sample(list(self.G.nodes), k_ev))

        # create one agent per node
        uid = 0
        for node in self.G.nodes:
            init_strategy = "C" if node in ev_nodes else "D"
            agent = EVAgent(uid, self, init_strategy)
            uid += 1
            self.schedule.add(agent)
            self.grid.place_agent(agent, node)

        self.datacollector = None
        if collect:
            self.datacollector = DataCollector(
                model_reporters={
                    "X": self.get_adoption_fraction,
                    "I": lambda m: m.infrastructure,
                },
                agent_reporters={"strategy": "strategy", "payoff": "payoff"},
            )

    def get_adoption_fraction(self):
        agents = self.schedule.agents
        if not agents:
            return 0.0
        return sum(1 for a in agents if a.strategy == "C") / len(agents)
    
    # ####################
    # Model step function
    #
    # The step function advances the model by one time step.
    # It first advances all agents, then computes the adoption fraction and infrastructure level.
    # The infrastructure level is updated based on the adoption fraction and the infrastructure growth rate.
    # The updated infrastructure level is clipped to the interval [0, 1].
    # Finally, if data collection is enabled, the model and agent data are collected.
    #######################
    
    def step(self): 
        self.schedule.step() # advance all agents

        X = self.get_adoption_fraction() # compute adoption fraction after all agents have advanced
        I = self.infrastructure # infrastructure level before this step
        dI = self.g_I * (X - I) # infrastructure growth rate, impacted by adoption fraction
        self.infrastructure = float(min(1.0, max(0.0, I + dI))) # clip infrastructure level to [0, 1]
    
        if self.datacollector is not None:
            self.datacollector.collect(self) # collect data at the end of each step
            
        self.step_count += 1 # increment step count after data collection

In [47]:

#########################
#
# Set initial adopters
# 
# Parameters
# - model: the EVStagHuntModel instance
# - X0_frac: fraction of agents to initially choose EV adoption
# - method: method to choose initial adopters ("random" or "degree")
# - seed: random seed for reproducibility
# - high: whether to choose high or low degree nodes for "degree" method
###########################
def set_initial_adopters(model, X0_frac, method="random", seed=None, high=True):
    """Set a fraction of agents to EV adopters using different heuristics."""
    rng = np.random.default_rng(seed)
    agents = model.schedule.agents
    n = len(agents)
    k = int(round(X0_frac * n))

    for a in agents:
        a.strategy = "D"

    if k <= 0:
        return

    if method == "random":
        idx = rng.choice(n, size=k, replace=False)
        for i in idx:
            agents[i].strategy = "C"
        return

    if method == "degree":
        deg = dict(model.G.degree())
        ordered_nodes = sorted(deg.keys(), key=lambda u: deg[u], reverse=high)
        chosen = set(ordered_nodes[:k])
        for a in agents:
            if a.unique_id in chosen:
                a.strategy = "C"
        return

    raise ValueError(f"Unknown method: {method}")




In [48]:
# -----------------------------
# Ratio sweep helpers (computation-only)
# -----------------------------
#########################
#
# Run a single network trial
# 
# Parameters
# - X0_frac: fraction of agents to initially choose EV adoption
# - ratio: payoff ratio between EV and DC agents (a0 = ratio*b - beta_I*I0)
# - I0: initial infrastructure level
# - beta_I: cost of EV adoption relative to DC (beta_I*I0)
# - b: payoff of EV (b)
# - g_I: infrastructure growth rate (g_I)
# - T: number of time steps to run
# - network_type: type of network to generate ("random" or "BA")
# - n_nodes: number of nodes in the network
# - p: probability of edge creation in random network
# - m: number of edges to attach from a new node to existing nodes in BA network
# - seed: random seed for reproducibility
# - tol: tolerance for convergence check (default: 1e-3)
# - patience: number of steps to wait for convergence (default: 30)

def run_network_trial(
    X0_frac: float,
    ratio: float,
    *,
    I0: float = 0.05,
    beta_I: float = 2.0,
    b: float = 1.0,
    g_I: float = 0.05,
    T: int = 200,
    network_type: str = "random",
    n_nodes: int = 120,
    p: float = 0.05,
    m: int = 2,
    seed: int | None = None,
    tol: float = 1e-3,
    patience: int = 30,
    collect: bool = False,
    strategy_choice_func: str = "imitate",
    tau: float = 1.0,
) -> float:
    """Run a single realisation and return final adoption fraction.

    Preserves the intended initial payoff ratio via a0 = ratio*b - beta_I*I0.
    Includes basic stability-based early stopping.
    """
    initial_ev = int(round(X0_frac * n_nodes))
    a0 = ratio * b - beta_I * I0

    model = EVStagHuntModel(
        initial_ev=initial_ev,
        a0=a0,
        beta_I=beta_I,
        b=b,
        g_I=g_I,
        I0=I0,
        seed=seed,
        network_type=network_type,
        n_nodes=n_nodes,
        p=p,
        m=m,
        collect=collect,
        strategy_choice_func=strategy_choice_func,
        tau=tau,
    )

    stable_steps = 0
    prev_X = None
    prev_I = None
    for _ in range(T):
        model.step()
        X = model.get_adoption_fraction()
        I = model.infrastructure
        if prev_X is not None and prev_I is not None:
            if abs(X - prev_X) < tol and abs(I - prev_I) < tol:
                stable_steps += 1
            else:
                stable_steps = 0
        prev_X, prev_I = X, I
        if X in (0.0, 1.0) and stable_steps >= 10:
            break
        if stable_steps >= patience:
            break

    return model.get_adoption_fraction()


In [49]:

#########################
#
# Compute final mean adoption fraction vs ratio
# 
##########################
def final_mean_adoption_vs_ratio(
    X0_frac: float,
    ratio_values: Iterable[float],
    *,
    I0: float = 0.05,
    beta_I: float = 2.0,
    b: float = 1.0,
    g_I: float = 0.05,
    T: int = 200,
    network_type: str = "random",
    n_nodes: int = 120,
    p: float = 0.05,
    m: int = 2,
    batch_size: int = 16,
    init_noise_I: float = 0.04,
    strategy_choice_func: str = "imitate",
    tau: float = 1.0,
) -> np.ndarray:
    """Compute mean final adoption across a sweep of ratio values.

    For each ratio, average over `batch_size` trials with jittered `I0` and seeds.
    Returns a numpy array of means aligned with `ratio_values` order.
    """
    ratios = list(ratio_values)
    means: List[float] = []
    for ratio in ratios:
        finals: List[float] = []
        for _ in range(batch_size):
            I0_j = float(np.clip(np.random.normal(loc=I0, scale=init_noise_I), 0.0, 1.0))
            seed_j = np.random.randint(0, 2**31 - 1)
            x_star = run_network_trial(
                X0_frac,
                ratio,
                I0=I0_j,
                beta_I=beta_I,
                b=b,
                g_I=g_I,
                T=T,
                network_type=network_type,
                n_nodes=n_nodes,
                p=p,
                m=m,
                seed=seed_j,
                collect=False,
                strategy_choice_func=strategy_choice_func,
                tau=tau,
            )
            finals.append(x_star)
        means.append(float(np.mean(finals)))
    return np.asarray(means, dtype=float)


In [51]:

#########################
#
# Compute heatmap row for a fixed ratio
# 
##########################
def _row_for_ratio_task(args: Dict) -> np.ndarray:
    """Top-level worker to compute one heatmap row for a fixed ratio.

    Returns an array of mean final adoption across provided X0_values.
    """
    ratio = args["ratio"]
    X0_values = args["X0_values"]
    I0 = args["I0"]
    beta_I = args["beta_I"]
    b = args["b"]
    g_I = args["g_I"]
    T = args["T"]
    network_type = args["network_type"]
    n_nodes = args["n_nodes"]
    p = args["p"]
    m = args["m"]
    batch_size = args["batch_size"]
    init_noise_I = args["init_noise_I"]
    strategy_choice_func = args["strategy_choice_func"]
    tau = args["tau"]

    row = np.empty(len(X0_values), dtype=float)
    for j, X0 in enumerate(X0_values):
        finals: List[float] = []
        for _ in range(batch_size):
            I0_j = float(np.clip(np.random.normal(loc=I0, scale=init_noise_I), 0.0, 1.0))
            seed_j = np.random.randint(0, 2**31 - 1)
            x_star = run_network_trial(
                X0_frac=X0,
                ratio=ratio,
                I0=I0_j,
                beta_I=beta_I,
                b=b,
                g_I=g_I,
                T=T,
                network_type=network_type,
                n_nodes=n_nodes,
                p=p,
                m=m,
                seed=seed_j,
                collect=False,
                strategy_choice_func=strategy_choice_func,
                tau=tau,
            )
            finals.append(x_star)
        row[j] = float(np.mean(finals))
    return row

    
#########################
#
# Compute heatmap matrix for phase sweep
# 
##########################
def phase_sweep_X0_vs_ratio(
    X0_values: Iterable[float],
    ratio_values: Iterable[float],
    *,
    I0: float = 0.05,
    beta_I: float = 2.0,
    b: float = 1.0,
    g_I: float = 0.05,
    T: int = 250,
    network_type: str = "BA",
    n_nodes: int = 120,
    p: float = 0.05,
    m: int = 2,
    batch_size: int = 16,
    init_noise_I: float = 0.04,
    strategy_choice_func: str = "logit",
    tau: float = 1.0,
    max_workers: int | None = None,
    backend: str = "process",
) -> np.ndarray:
    """Compute a heatmap matrix of mean final adoption X* over (X0, ratio).

    Returns an array of shape (len(ratio_values), len(X0_values)) aligned with
    the provided orders. Rows correspond to ratios; columns to X0 values.
    """
    X0_values = list(X0_values)
    ratio_values = list(ratio_values)
    X_final = np.zeros((len(ratio_values), len(X0_values)), dtype=float)

    # Prepare tasks per ratio
    tasks: List[Dict] = []
    for ratio in ratio_values:
        tasks.append({
            "ratio": ratio,
            "X0_values": X0_values,
            "I0": I0,
            "beta_I": beta_I,
            "b": b,
            "g_I": g_I,
            "T": T,
            "network_type": network_type,
            "n_nodes": n_nodes,
            "p": p,
            "m": m,
            "batch_size": batch_size,
            "init_noise_I": init_noise_I,
            "strategy_choice_func": strategy_choice_func,
            "tau": tau,
        })

    if max_workers is None:
        try:
            max_workers = os.cpu_count() or 1
        except Exception:
            max_workers = 1

    Executor = ProcessPoolExecutor if backend == "process" and max_workers > 1 else ThreadPoolExecutor
    if max_workers > 1:
        with Executor(max_workers=max_workers) as ex:
            futures = [ex.submit(_row_for_ratio_task, args) for args in tasks]
            for i, fut in enumerate(futures):
                row = fut.result()
                X_final[i, :] = row
    else:
        for i, args in enumerate(tasks):
            row = _row_for_ratio_task(args)
            X_final[i, :] = row

    return X_final

In [52]:
model = EVStagHuntModel(
    initial_ev=10,
    a0=10.0,
    beta_I=3.0,
    b=10.0,
    g_I=0.5,
    I0=0.05,
    n_nodes=100,
    network_type="random",
    collect=True,
    strategy_choice_func="imitate"
)

set_initial_adopters(model, X0_frac=0.3, method="degree", high=True)

for _ in range(100):
    model.step()

print("Final adoption:", model.get_adoption_fraction())
print("Final infrastructure:", model.infrastructure)

df = model.datacollector.get_model_vars_dataframe()
print(df)

model.step()
print([a.next_strategy for a in model.schedule.agents][:10])

Final adoption: 0.0
Final infrastructure: 8.04638123325432e-31
       X             I
0   0.23  1.400000e-01
1   0.17  1.550000e-01
2   0.08  1.175000e-01
3   0.01  6.375000e-02
4   0.00  3.187500e-02
..   ...           ...
95  0.00  1.287421e-29
96  0.00  6.437105e-30
97  0.00  3.218552e-30
98  0.00  1.609276e-30
99  0.00  8.046381e-31

[100 rows x 2 columns]
['D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D']


In [53]:
model.step()
print([a.strategy for a in model.schedule.agents][:10])
print([a.next_strategy for a in model.schedule.agents][:10])

['D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D']
['D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D']


In [58]:
for _ in range(10):
    model.step()

print([a.strategy for a in model.schedule.agents][:20])
print("Adoption fraction:", model.get_adoption_fraction())

print([a.next_strategy for a in model.schedule.agents][:20])
print("Adoption fraction:", model.get_adoption_fraction())

['D', 'D', 'D', 'D', 'D', 'D', 'C', 'D', 'C', 'D', 'D', 'D', 'D', 'D', 'D', 'C', 'D', 'D', 'D', 'C']
Adoption fraction: 0.3
['D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'C', 'D', 'D', 'D']
Adoption fraction: 0.3


In [54]:
model = EVStagHuntModel(
    initial_ev=10,
    a0=10.0,
    beta_I=3.0,
    b=10.0,
    g_I=0.5,
    I0=0.05,
    n_nodes=50,
    network_type="random",
    collect=True,
    strategy_choice_func="logit",  # ← change here
    tau=0.5                        # ← moderate exploration
)

set_initial_adopters(model, X0_frac=0.3, method="degree", high=True)

for _ in range(10000):  # fewer steps often enough
    model.step()

print("Adoption fraction:", model.get_adoption_fraction())


print([a.strategy for a in model.schedule.agents][:20])
print("Adoption fraction:", model.get_adoption_fraction())

print([a.next_strategy for a in model.schedule.agents][:20])
print("Adoption fraction:", model.get_adoption_fraction())

for _ in range(30):
    model.step()
print("  step_count:", model.step_count)
print("  adoption fraction X:", model.get_adoption_fraction())
print("  infrastructure I:", model.infrastructure)
print("")

KeyboardInterrupt: 

In [55]:
# -----------------------------
# Laatste cel: alles aanroepen
# (Plak dit precies als laatste cel; ik wijzig geen eerdere cellen)
# -----------------------------

def main():
    import traceback
    print("Start quick checks of model functions...\n")

    try:
        # 1) Kleine model-run (sanity)
        print("1) Kleine EVStagHuntModel run (50 nodes, 30 stappen)...")
        model = EVStagHuntModel(
            initial_ev=5,
            a0=2.0,
            beta_I=2.0,
            b=1.0,
            g_I=0.05,
            I0=0.05,
            seed=42,
            network_type="random",
            n_nodes=50,
            p=0.05,
            m=2,
            collect=True,
            strategy_choice_func="imitate",
            tau=1.0,
        )
        
        # Zorg dat er een kleine fractie adoptanten is (optioneel)
        set_initial_adopters(model, X0_frac=0.25, method="random", seed=42)

        for _ in range(30):
            model.step()
        print("  step_count:", model.step_count)
        print("  adoption fraction X:", model.get_adoption_fraction())
        print("  infrastructure I:", model.infrastructure)
        print("")

        # 2) Single network trial (run_network_trial)
        print("2) Single network trial (run_network_trial)...")
        x_star = run_network_trial(
            X0_frac=0.25,
            ratio=1.5,
            I0=0.05,
            beta_I=2.0,
            b=5.0,
            g_I=0.05,
            T=200,
            network_type="random",
            n_nodes=30,
            p=0.05,
            m=2,
            seed=123,
            tol=1e-3,
            patience=10,
            collect=False,
            strategy_choice_func="imitate",
            tau=1.0,
        )
        print("  final adoption from single trial:", x_star)
        print("")

        # 3) Small ratio sweep (final_mean_adoption_vs_ratio)
        # Let op: zet batch_size klein voor snelheid
        print("3) Kleine ratio sweep (final_mean_adoption_vs_ratio) — kort, batch_size=4 ...")
        ratios = [0.8, 1.0, 1.2, 1.5]
        means = final_mean_adoption_vs_ratio(
            X0_frac=0.25,
            ratio_values=ratios,
            I0=0.05,
            beta_I=0.5,
            b=1.0,
            g_I=0.1,
            T=100,
            network_type="random",
            n_nodes=100,
            p=0.05,
            m=2,
            batch_size=4,          # klein om snel te draaien in notebook
            init_noise_I=0.01,
            strategy_choice_func="imitate",
            tau=1.0,
        )
        print("  ratio -> mean final adoption:")
        for r, mval in zip(ratios, means.tolist()):
            print(f"    {r:>4} -> {mval:.4f}")
        print("")

        # 4) Row-for-ratio worker example (_row_for_ratio_task)
        print("4) Voorbeeld _row_for_ratio_task (korte X0 lijst, batch_size=3)...")
        args = {
            "ratio": 1.2,
            "X0_values": [0.0, 0.05, 0.1],
            "I0": 0.05,
            "beta_I": 2.0,
            "b": 1.0,
            "g_I": 0.1,
            "T": 100,
            "network_type": "random",
            "n_nodes": 100,
            "p": 0.05,
            "m": 2,
            "batch_size": 3,        # klein om snel te doen
            "init_noise_I": 0.01,
            "strategy_choice_func": "imitate",
            "tau": 1.0,
        }
        row = _row_for_ratio_task(args)
        print("  row (mean final adoption for each X0):", row.tolist())
        print("")

        result_summary = {
            "model_adoption": model.get_adoption_fraction(),
            "single_trial": x_star,
            "ratio_sweep": {float(r): float(v) for r, v in zip(ratios, means.tolist())},
            "row_example": [float(v) for v in row.tolist()],
        }

        print("All checks finished successfully.")
        return result_summary

    except Exception as exc:
        print("Er trad een fout op tijdens het uitvoeren van de checks:")
        traceback.print_exc()
        raise

# In Jupyter is het gebruikelijk om main() direct te draaien:
results = main()
print("\nSamenvatting resultaten:")
print(results)

Start quick checks of model functions...

1) Kleine EVStagHuntModel run (50 nodes, 30 stappen)...
  step_count: 30
  adoption fraction X: 0.12
  infrastructure I: 0.09708522195408047

2) Single network trial (run_network_trial)...
  final adoption from single trial: 0.03333333333333333

3) Kleine ratio sweep (final_mean_adoption_vs_ratio) — kort, batch_size=4 ...
  ratio -> mean final adoption:
     0.8 -> 0.0050
     1.0 -> 0.0050
     1.2 -> 0.0025
     1.5 -> 0.0050

4) Voorbeeld _row_for_ratio_task (korte X0 lijst, batch_size=3)...
  row (mean final adoption for each X0): [0.0, 0.0, 0.0]

All checks finished successfully.

Samenvatting resultaten:
{'model_adoption': 0.12, 'single_trial': 0.03333333333333333, 'ratio_sweep': {0.8: 0.005, 1.0: 0.005, 1.2: 0.0025, 1.5: 0.005}, 'row_example': [0.0, 0.0, 0.0]}
